In [25]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
import time

# 需要什么读取什么不要一次性全都读取了，影响程序速度

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

time1 = time.time()

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)
    

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)

    
# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()

# 提取B区信息
B_DF = DTBS_DataFrame['B']
# 计算所有转债每天的涨跌幅，并找出涨幅大于前N个交易日的q分位数涨跌幅的均值的转债

In [26]:
A_DF['双低+平底溢价率'] = A_DF['dl'] + A_DF['pd']

# DataFrame新增列用于存放前一天收盘价、涨跌幅
A_DF.sort_values(by = ['bond_code','date'], inplace=True, ascending=[True,True])
A_DF['lastday_dp'] = A_DF.groupby('bond_code')['dp'].shift(1)
A_DF['change'] = A_DF['dp'] / A_DF['lastday_dp'] - 1
A_DF = pd.merge(A_DF, B_DF[['bond_code', 'sn']].drop_duplicates(), on='bond_code', how='left')

# 动量因子
n = 10
A_DF.sort_values(['bond_code', 'date'], ascending=[True,True], inplace=True)
grouped = A_DF.groupby('bond_code')
A_DF['前n天涨跌幅的平均值'] = grouped['change'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].mean(), raw=True).reset_index(level=0, drop=True)

# 双低+平底溢价率位于每天的前  q_dl_add_pd  分位
q_dl_add_pd = 0.3
A_DF['quantile_双低+平底溢价率'] = A_DF.groupby('date')['双低+平底溢价率'].transform(lambda x: x.dropna().quantile(q_dl_add_pd))
A_DF['当日是否双低+平底溢价率的值较小'] = (A_DF['双低+平底溢价率'] <= A_DF['quantile_双低+平底溢价率']).astype(int)

In [27]:
# 获取到目前为止已退市可转债代码，这里的目的是希望在买入时规避掉这些即将退市的转债
delisted_bond_codes = list(B_DF.loc[B_DF['dld'] < '2046-01-01', 'bond_code'])
A_DF['是否属于已退市转债的最后N个交易日（1代表是）'] = 0

A_DF_filtered = A_DF[A_DF['bond_code'].isin(delisted_bond_codes)].copy()

# 然后，按"A"列进行分组，并在"B"列中找出每个组的topN的值
N = 20
top_n = A_DF_filtered.groupby('bond_code')['date'].apply(lambda x: x.sort_values(ascending=False).head(N))

# 然后，你可以根据筛选结果，给"C"列赋值
for index in top_n.index:
    A_DF_filtered.loc[index[1], '是否属于已退市转债的最后N个交易日（1代表是）'] = 1

# 将修改过的部分合并到原始的df中
A_DF.update(A_DF_filtered)
print(A_DF)

        bond_code        date      cpr       dp        bl         trt     ytm  \
0       110030.SH  2017-01-03  41.5266  114.040  9.780730    0.373183 -1.6776   
1       110030.SH  2017-01-04  40.3232  114.230  9.780730    0.122792 -1.7348   
2       110030.SH  2017-01-05  40.2584  114.950  9.780730    1.022316 -1.9463   
3       110030.SH  2017-01-06  41.3362  114.860  9.780730    0.493010 -1.9219   
4       110030.SH  2017-01-09  42.6442  115.530  9.780730    0.341181 -2.1219   
...           ...         ...      ...      ...       ...         ...     ...   
391526  128145.SZ  2023-02-20  -0.7001  131.670  1.187342  214.411349 -3.3080   
391527  128145.SZ  2023-02-21  -0.3236  130.449  1.099123   82.549997 -3.0847   
391528  128145.SZ  2023-02-22  -0.5313  127.030  0.976149   59.889218 -2.4404   
391529  128145.SZ  2023-02-23  -0.2378  125.205  0.773509   68.496036 -2.0877   
391530  128145.SZ  2023-02-24  -0.3382  122.499  0.156706  488.807066 -1.5511   

              yl        dl 

In [28]:
# 添加若干限定条件
empty_df_select = A_DF[(A_DF['当日是否双低+平底溢价率的值较小']==1) & 
                       (A_DF['date'] >= '2022-01-01') &
                       (A_DF['date'] <= '2023-08-20')].copy()
# print(empty_df_select)

empty_df_select.sort_values(['date','前n天涨跌幅的平均值'],ascending=[True,False] ,inplace=True)
empty_df_select = empty_df_select.groupby(['date']).head(5)


# 最后需要展示的列
output_columns = ['date' ,'bond_code' ,'dp','sn','change', 'dl','cpr','bl','yl','trt', 'ytm','qs','qs30','前n天涨跌幅的平均值','双低+平底溢价率']

empty_df_select = empty_df_select[empty_df_select['是否属于已退市转债的最后N个交易日（1代表是）'] == 0][output_columns].copy()


"""保存到EXCEL得时候重命名列名，便于阅读"""
empty_df_select.rename(columns={'date' : '日期',
                                'bond_code' : '可转债代码',
                                'dp' : '价格（收盘价全价）',
                                'cpr' : '转股溢价率',
                                'dl' : '双低',
                                'change':'涨跌幅',
                                'sn' : '正股名称',
                                'bl' : '债券余额（亿）',
                                'yl' : '剩余期限（年）',
                                'trt' : '换手率',
                                'ytm' : '到期收益率'},inplace=True)

empty_df_select.sort_values(by=['日期','可转债代码'],ascending=[True,True] ,inplace=True)

time2 = time.time()

empty_df_select.to_excel('pre_data_for_backtest\\双低+平底溢价率_前n天涨跌幅平均值（全量数据DTBS，用于回测）.xlsx',index=False)